# CS4001/4042 Assignment 1, Part B, Q2
In Question B1, we used the Category Embedding model. This creates a feedforward neural network in which the categorical features get learnable embeddings. In this question, we will make use of a library called Pytorch-WideDeep. This library makes it easy to work with multimodal deep-learning problems combining images, text, and tables. We will just be utilizing the deeptabular component of this library through the TabMlp network:

In [ ]:
!pip install pytorch-widedeep

In [1]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.models import TabMlp, WideDeep
from pytorch_widedeep import Trainer
from pytorch_widedeep.metrics import R2Score

>Divide the dataset (‘hdb_price_prediction.csv’) into train and test sets by using entries from the year 2020 and before as training data, and entries from 2021 and after as the test data.

In [2]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here
df_train = df[df['year'] <= 2020]
df_test = df[df['year'] >= 2021]
df_test

,month,year,town,full_address,nearest_stn,dist_to_nearest_stn,dist_to_dhoby,degree_centrality,eigenvector_centrality,flat_model_type,remaining_lease_years,floor_area_sqm,storey_range,resale_price
87370,1,2021,ANG MO KIO,170 ANG MO KIO AVENUE 4,Yio Chu Kang,1.276775,8.339960,0.016807,0.002459,"2 ROOM, Improved",64.083333,45.0,01 TO 03,211000.0
87371,1,2021,ANG MO KIO,170 ANG MO KIO AVENUE 4,Yio Chu Kang,1.276775,8.339960,0.016807,0.002459,"2 ROOM, Improved",64.083333,45.0,07 TO 09,225000.0
87372,1,2021,ANG MO KIO,331 ANG MO KIO AVENUE 1,Ang Mo Kio,0.884872,6.981730,0.016807,0.006243,"3 ROOM, New Generation",59.000000,68.0,04 TO 06,260000.0
87373,1,2021,ANG MO KIO,534 ANG MO KIO AVENUE 10,Ang Mo Kio,0.677246,8.333056,0.016807,0.006243,"3 ROOM, New Generation",58.166667,68.0,04 TO 06,265000.0
87374,1,2021,ANG MO KIO,561 ANG MO KIO AVENUE 10,Ang Mo Kio,0.922047,8.009223,0.016807,0.006243,"3 ROOM, New Generation",58.083333,68.0,01 TO 03,265000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159548,8,2023,YISHUN,344 YISHUN AVENUE 11,Yishun,1.032889,14.360442,0.016807,0.000382,"EXECUTIVE, Maisonette",64.083333,145.0,01 TO 03,780000.0
159549,8,2023,YISHUN,325 YISHUN CENTRAL,Yishun,0.791756,14.388222,0.016807,0.000382,"EXECUTIVE, Maisonette",64.333333,146.0,10 TO 12,808000.0
159550,8,2023,YISHUN,325 YISHUN CENTRAL,Yishun,0.791756,14.388222,0.016807,0.000382,"EXECUTIVE, Maisonette",64.333333,146.0,07 TO 09,788888.0
159551,8,2023,YISHUN,387 YISHUN RING ROAD,Yishun,1.361636,14.417476,0.016807,0.000382,"EXECUTIVE, Apartment",63.916667,142.0,10 TO 12,822800.0


>Refer to the documentation of Pytorch-WideDeep and perform the following tasks:
https://pytorch-widedeep.readthedocs.io/en/latest/index.html
* Use [**TabPreprocessor**](https://pytorch-widedeep.readthedocs.io/en/latest/examples/01_preprocessors_and_utils.html#2-tabpreprocessor) to create the deeptabular component using the continuous
features and the categorical features. Use this component to transform the training dataset.
* Create the [**TabMlp**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/model_components.html#pytorch_widedeep.models.tabular.mlp.tab_mlp.TabMlp) model with 2 linear layers in the MLP, with 200 and 100 neurons respectively.
* Create a [**Trainer**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/trainer.html#pytorch_widedeep.training.Trainer) for the training of the created TabMlp model with the root mean squared error (RMSE) cost function. Train the model for 100 epochs using this trainer, keeping a batch size of 64. (Note: set the *num_workers* parameter to 0.)

In [3]:
# TODO: Enter your code here
import torch
import torch.nn as nn
continuous_cols = ["dist_to_nearest_stn", "dist_to_dhoby", "degree_centrality", "eigenvector_centrality", "remaining_lease_years", "floor_area_sqm"]
cat_cols = [
    "month",
    "town",
    "flat_model_type",
    "storey_range",
]

tab_preprocessor = TabPreprocessor(
    cat_embed_cols=cat_cols, continuous_cols=continuous_cols
)

X_tab = tab_preprocessor.fit_transform(df_train)

tab_mlp_model = TabMlp(mlp_hidden_dims=[200,100], column_idx=tab_preprocessor.column_idx, cat_embed_input=tab_preprocessor.cat_embed_input, continuous_cols=tab_preprocessor.continuous_cols)
model = WideDeep(deeptabular=tab_mlp_model)

trainer = Trainer(
    model = model,
    objective = "root_mean_squared_error",
    num_workers = 0
)
y = df_train['resale_price'].to_numpy()
trainer.fit(X_tab = X_tab, target= y, n_epochs=100, batch_size=64)

c:\Users\mdzak\anaconda3\envs\nnpartb2\lib\site-packages\pytorch_widedeep\preprocessing\tab_preprocessor.py:334: UserWarning: Continuous columns will not be normalised
  warnings.warn("Continuous columns will not be normalised")
epoch 100: 100%|██████████| 1366/1366 [00:06<00:00, 195.74it/s, loss=4.96e+4]


>Report the test RMSE and the test R2 value that you obtained.

In [4]:
# TODO: Enter your code here
def r_squared_numpy(y_true, y_pred):
    y_mean = np.mean(y_true)
    ss_tot = np.sum((y_true - y_mean)**2)
    ss_res = np.sum((y_true - y_pred)**2)
    r2 = 1 - (ss_res / ss_tot)
    return r2
def calculate_rmse(y_true, y_pred):
    squared_diff = (y_true - y_pred) ** 2
    mean_squared_diff = np.mean(squared_diff)
    rmse = np.sqrt(mean_squared_diff)
    return rmse

X_tab_test = tab_preprocessor.transform(df_test)
y_test = df_test['resale_price'].to_numpy()
pred = trainer.predict(X_tab = X_tab_test, batch_size=64)

rmse = calculate_rmse(y_test, pred)
r2 = r_squared_numpy(y_test, pred)

print("RMSE: ", rmse)
print("R2: ", r2)

predict: 100%|██████████| 1128/1128 [00:02<00:00, 499.85it/s]

RMSE:  96925.95585339662
R2:  0.6717696655512904
